In [1]:
require 'torch'
require 'nn'
require 'optim'
mnist = require 'mnist'
fullset = mnist.traindataset()
testset = mnist.testdataset()
--[[for i=1,fullset.size do
    itorch.image(fullset.data[i])
    print(fullset.label[i])
    end--]]
trainset = {
    size = 50000,
    data = fullset.data[{{1,50000}}]:double(),
    label = fullset.label[{{1,50000}}]
}
validationset = {
    size = 10000,
    data = fullset.data[{{50001,60000}}]:double(),
    label = fullset.label[{{50001,60000}}]
}

model = nn.Sequential()
model:add(nn.Reshape(28*28))
model:add(nn.Linear(28*28, 30))
model:add(nn.Tanh())
model:add(nn.Linear(30, 10))
model:add(nn.LogSoftMax())
criterion = nn.ClassNLLCriterion()

sgd_params = {
   learningRate = 1e-2,
   learningRateDecay = 1e-4,
   weightDecay = 1e-3,
   momentum = 1e-4
}
x, dl_dx = model:getParameters()
step = function(batch_size)
    local current_loss = 0
    local count = 0
    local shuffle = torch.randperm(trainset.size)
    batch_size = batch_size or 200
    
    for t = 1,trainset.size,batch_size do
        -- setup inputs and targets for this mini-batch
        local size = math.min(t + batch_size - 1, trainset.size) - t
        local inputs = torch.Tensor(size, 28, 28)
        local targets = torch.Tensor(size)
        for i = 1,size do
            local input = trainset.data[shuffle[i+t]]
            local target = trainset.label[shuffle[i+t]]
            -- if target == 0 then target = 10 end
            inputs[i] = input
            targets[i] = target
        end
        targets:add(1)
        
        local feval = function(x_new)
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero()

            -- perform mini-batch gradient descent
            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        current_loss = current_loss + fs[1]
    end

    -- normalize loss
    return current_loss / count
end


eval = function(dataset, batch_size)
    local count = 0
    batch_size = batch_size or 200
    
    for i = 1,dataset.size,batch_size do
        local size = math.min(i + batch_size - 1, dataset.size) - i
        local inputs = dataset.data[{{i,i+size-1}}]
        local targets = dataset.label[{{i,i+size-1}}]:long()
        local outputs = model:forward(inputs)
        local _, indices = torch.max(outputs, 2)
        indices:add(-1)
        local guessed_right = indices:eq(targets):sum()
        count = count + guessed_right
    end

    return count / dataset.size
end

max_iters = 30
do
    local last_accuracy = 0
    local decreasing = 0
    local threshold = 1 -- how many deacreasing epochs we allow
    for i = 1,max_iters do
        local loss = step()
        print(string.format('Epoch: %d Current loss: %4f', i, loss))
        local accuracy = eval(validationset)
        print(string.format('Accuracy on the validation set: %4f', accuracy))
        if accuracy < last_accuracy then
            if decreasing > threshold then break end
            decreasing = decreasing + 1
        else
            decreasing = 0
        end
        last_accuracy = accuracy
    end
end

Epoch: 1 Current loss: 1.166684	


Accuracy on the validation set: 0.855100	


Epoch: 2 Current loss: 0.665227	


Accuracy on the validation set: 0.883500	


Epoch: 3 Current loss: 0.536459	


...

Accuracy on the validation set: 0.923500	


Epoch: 27 Current loss: 0.276376	


Accuracy on the validation set: 0.921000	


Epoch: 28 Current loss: 0.270281	


Accuracy on the validation set: 0.918800	


In [2]:
testset.data = testset.data:double()

In [3]:
eval(testset)

0.9156	


In [4]:
paths = require 'paths'

In [6]:
filename = paths.concat(paths.cwd(), 'model.net')

In [7]:
filename

/home/iop/model.net	


In [8]:
help(torch.save)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
torch.save(filename, object [, format, referenced])


Writes object into a file named filename . The format can be set 
to ascii or binary (default is binary). Binary format is platform
dependent, but typically more compact and faster to read/write. The ASCII
format is platform-independent, and should be used to share data structures
across platforms. The option referenced specifies if
object references should be tracked or not
( true by default).
 -- arbitrary object:
obj = {
   mat = torch.randn(10,10),
   name = '10',
   test = {
      entry = 1
   }
}
-- save to disk:
torch.save('test.dat', obj) 	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	



In [9]:
torch.save(filename, model)

In [10]:
help(torch.load)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	
[object] torch.load(filename [, format, referenced])


Reads object from a file named filename .
The format can be set to ascii , binary , b32 or b64 (default 
is binary).
Binary format is platform dependent, but typically more compact and faster 
to read/write.
Use b32 / b64 , instead of binary , for loading files saved on a 
32/64 bit OS.
The ASCII format is platform-independent, and may be used to share data 
structures across platforms.
The option referenced specifies if object references should be tracked 
or not ( true by default).
Note that files written with referenced at true cannot be loaded with 
referenced at false .
 -- given serialized object from section above, reload:
obj = torch.load('test.dat')
print(obj)
-- will print:
-- {[mat]  = DoubleTensor - size: 10x10
--  [name] = string : &quot;10&quot;
--  [test] = table - size: 0} 	
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++	



In [16]:
eval1 = function(dataset)
   local count = 0
   for i = 1,dataset.size do
      local output = model1:forward(dataset.data[i])
      local _, index = torch.max(output, 1) -- max index
      local digit = index[1] % 10
      if digit == dataset.label[i] then count = count + 1 end
   end

   return count / dataset.size
end

In [17]:
eval1(testset)

[string "eval1 = function(dataset)..."]:4: attempt to index global 'model1' (a nil value)
stack traceback:
	[string "eval1 = function(dataset)..."]:4: in function 'f'
	[string "local f = function() return eval1(testset) en..."]:1: in main chunk
	[C]: in function 'xpcall'
	/home/iop/torch/install/share/lua/5.1/itorch/main.lua:209: in function </home/iop/torch/install/share/lua/5.1/itorch/main.lua:173>
	/home/iop/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/home/iop/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/home/iop/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/home/iop/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/iop/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00405d50: 